In [1]:
import os
import time
import sys

import vectorbt as vbt
import pandas as pd

In [2]:
price = vbt.YFData.download('BTC-USD', start='2018-01-01', end = '2024-01-01')
price

In [14]:
st = time.time()
close = price.get('Close')
fast_ma = vbt.MA.run(close, 50, short_name='fast_ma')
slow_ma = vbt.MA.run(close, 200, short_name='slow_ma')
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)
pf = vbt.Portfolio.from_signals(close, entries, exits, fees=0.005)
et = time.time()
print(f'Elapsed time: {1000*(et-st):.2f} ms')

Elapsed time: 50.77 ms


In [1]:
#pf.orders.records_readable

In [19]:
# save price data to file
df

Date
2018-01-01 00:00:00+00:00    13657.200195
2018-01-02 00:00:00+00:00    14982.099609
2018-01-03 00:00:00+00:00    15201.000000
2018-01-04 00:00:00+00:00    15599.200195
2018-01-05 00:00:00+00:00    17429.500000
                                 ...     
2023-12-28 00:00:00+00:00    42627.855469
2023-12-29 00:00:00+00:00    42099.402344
2023-12-30 00:00:00+00:00    42156.902344
2023-12-31 00:00:00+00:00    42265.187500
2024-01-01 00:00:00+00:00    44167.332031
Freq: D, Name: Close, Length: 2192, dtype: float64

In [21]:
df= price.data['BTC-USD']["Close"]
df.to_csv('btc_usd.csv')